# Library import

In [1]:
import cv2
print(cv2.__version__)
print(hasattr(cv2, 'ximgproc'))


4.10.0
True


In [2]:
# 필요 library들을 import합니다.
import os
from typing import Tuple, Any, Callable, List, Optional, Union

import cv2
import timm
import torch
import random
import numpy as np
import pandas as pd
import albumentations as A
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt

# from torchcam.methods import GradCAM
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

/opt/conda/envs/blueberry/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(torch.cuda.is_available())

True


In [4]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Dataset Class

## BPD & MSR

In [5]:
import cv2
import numpy as np

def bezier_pivot_deformation(image, num_deformations=6, deformation_degree=10): # 원본 이미지로부터 6개의 변형된 이미지를 생성
    """
    Applies Bezier pivot based deformation (BPD) on a sketch image.

    Args:
    - image (numpy.ndarray): The input sketch image.
    - num_deformations (int): Number of deformations to generate.
    - deformation_degree (int): The degree of random shift for control pivots.
    
    Returns:
    - List of deformed images.
    """
    def fit_bezier_curve(points):
        # Fit a cubic Bezier curve to the points using least squares method
        n = len(points)
        t = np.linspace(0, 1, n)
        phi = 1 - t
        A = np.vstack([phi**3, 3*t*phi**2, 3*t**2*phi, t**3]).T
        B = np.linalg.lstsq(A, points, rcond=None)[0]
        return B

    def deform_curve(p0, p1, p2, p3, alpha=deformation_degree):
        # Randomly deform the control pivots
        delta = np.random.uniform(-alpha, alpha, size=(2,))
        return p0, p1 + delta, p2 + delta, p3

    # Step 1: Convert image to binary and skeletonize
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    skeleton = cv2.ximgproc.thinning(binary)

    # Step 2: Segment into patches and deform curves
    patches = []
    patch_size = 32
    h, w = skeleton.shape
    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = skeleton[y:y + patch_size, x:x + patch_size]
            patches.append(patch)

    deformed_images = []
    for _ in range(num_deformations):
        new_image = np.zeros_like(skeleton)
        for patch in patches:
            # Fit Bezier curve to patch
            y, x = np.where(patch > 0)
            if len(x) > 3:
                points = np.column_stack((x, y))
                p0, p1, p2, p3 = fit_bezier_curve(points)

                # Deform curve using random shifts
                p0, p1, p2, p3 = deform_curve(p0, p1, p2, p3)

                # Reconstruct the patch with deformed curve
                new_image[y, x] = 255
        deformed_images.append(new_image)

    return deformed_images


In [6]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.cluster import KMeans

# 패치 추출 및 HOG 특징 계산 함수
def extract_patches_and_hog_features(image, patch_size=31):
    """
    주어진 이미지에서 패치를 추출하고, 각 패치의 HOG 특징을 계산하는 함수.
    
    Args:
    - image (numpy.ndarray): 입력 이미지.
    - patch_size (int): 패치의 크기.
    
    Returns:
    - patches (List): 패치의 리스트.
    - hog_features (List): HOG 특징의 리스트.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    skeleton = cv2.ximgproc.thinning(binary)

    h, w = skeleton.shape
    patches = []
    hog_features = []
    
    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = skeleton[y:y + patch_size, x:x + patch_size]
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:  # 패치 크기 확인
                patches.append((patch, (x, y)))
                hog_feature = hog(patch, pixels_per_cell=(8, 8), cells_per_block=(1, 1), feature_vector=True)
                hog_features.append(hog_feature)
    
    return patches, hog_features

# 클러스터 중심 계산 함수
def compute_cluster_centers(images, n_clusters=150, patch_size=31):
    """
    여러 이미지에서 HOG 특징을 추출하고, 이를 KMeans로 클러스터링하여 클러스터 중심을 계산.
    
    Args:
    - images (List of np.ndarray): 학습 이미지들의 리스트.
    - n_clusters (int): 클러스터 개수.
    - patch_size (int): 각 패치의 크기.
    
    Returns:
    - cluster_centers (np.ndarray): 계산된 클러스터 중심.
    """
    all_hog_features = []
    
    # 모든 이미지에서 패치와 HOG 특징 추출
    for image in images:
        _, hog_features = extract_patches_and_hog_features(image, patch_size)
        all_hog_features.extend(hog_features)
    
    # KMeans로 클러스터링
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(all_hog_features)
    
    return kmeans.cluster_centers_

# MSR 함수
def mean_stroke_reconstruction(image, cluster_centers, patch_size=31):
    """
    계산된 클러스터 중심을 사용하여 입력 이미지에 Mean Stroke Reconstruction을 적용하는 함수.
    
    Args:
    - image (numpy.ndarray): 입력 스케치 이미지.
    - cluster_centers (List): 미리 계산된 클러스터 중심 리스트.
    - patch_size (int): 패치 크기.
    
    Returns:
    - Reconstructed image (numpy.ndarray): 재구성된 이미지.
    """
    patches, hog_features = extract_patches_and_hog_features(image, patch_size)
    
    # 각 패치를 클러스터 중심으로 대체
    kmeans = KMeans(n_clusters=len(cluster_centers)).fit(hog_features)
    labels = kmeans.predict(hog_features)

    new_image = np.zeros_like(patches[0][0])
    
    for i, (patch, (x, y)) in enumerate(patches):
        mean_stroke = cluster_centers[labels[i]]
        new_image[y:y + patch_size, x:x + patch_size] = np.reshape(mean_stroke, (patch_size, patch_size))

    return new_image


In [7]:
def apply_bpd_with_partial_msr(image, cluster_centers, msr_ratio=0.5, num_deformations=6):
    """
    BPD로 변형된 이미지 중 일부에만 MSR을 적용하는 함수.
    
    Args:
    - image (numpy.ndarray): 원본 스케치 이미지.
    - cluster_centers (List): MSR에서 사용할 클러스터 중심.
    - msr_ratio (float): MSR을 적용할 이미지 비율 (0.0 ~ 1.0).
    - num_deformations (int): BPD로 생성할 이미지 개수.
    
    Returns:
    - List of transformed images, where a portion of them have MSR applied.
    """
    # 1. BPD를 적용하여 여러 개의 변형된 이미지 생성
    deformed_images = bezier_pivot_deformation(image, num_deformations=num_deformations)
    
    # 2. MSR 적용 여부 결정
    num_msr = int(msr_ratio * num_deformations)  # MSR을 적용할 이미지 개수
    msr_images = random.sample(deformed_images, num_msr)  # 무작위로 선택
    
    final_images = []
    for img in deformed_images:
        # NumPy 배열 비교 수정
        if any(np.array_equal(img, msr_img) for msr_img in msr_images):
            # MSR 적용
            img = mean_stroke_reconstruction(img, cluster_centers)
        final_images.append(img)
    
    return final_images

In [8]:
class CustomDataset(Dataset):
    def __init__(
        self, 
        root_dir: str, 
        info_df: pd.DataFrame, 
        transform: Callable,
        cluster_centers: np.ndarray,  # MSR에 사용할 클러스터 중심 추가
        is_inference: bool = False,
        apply_bpd_msr: bool = False  # BPD와 MSR 적용 여부
    ):
        # 데이터셋의 기본 경로, 이미지 변환 방법, 이미지 경로 및 레이블을 초기화합니다.
        self.root_dir = root_dir  # 이미지 파일들이 저장된 기본 디렉토리
        self.transform = transform  # 이미지에 적용될 변환 처리
        self.cluster_centers = cluster_centers  # MSR에 사용할 클러스터 중심
        self.is_inference = is_inference # 추론인지 확인
        self.apply_bpd_msr = apply_bpd_msr  # BPD와 MSR 적용 여부
        self.image_paths = info_df['image_path'].tolist()  # 이미지 파일 경로 목록
        
        if not self.is_inference:
            self.targets = info_df['target'].tolist()  # 각 이미지에 대한 레이블 목록

    def __len__(self) -> int:
        # 데이터셋의 총 이미지 수를 반환합니다.
        return len(self.image_paths)

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, int], torch.Tensor]:
        # 주어진 인덱스에 해당하는 이미지를 로드하고 변환을 적용한 후, 이미지와 레이블을 반환합니다.
        img_path = os.path.join(self.root_dir, self.image_paths[index])  # 이미지 경로 조합
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)  # 이미지를 BGR 컬러 포맷의 numpy array로 읽어옵니다.
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR 포맷을 RGB 포맷으로 변환합니다.
        
        if len(image.shape) == 2:  # 1채널 이미지일 경우
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # 1채널 이미지를 3채널로 변환
        else:  # 3채널 이미지일 경우
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
        
        
        # BPD와 MSR을 적용하는 부분 추가
        if self.apply_bpd_msr:
            image = apply_bpd_with_partial_msr(
                image, 
                self.cluster_centers, 
                msr_ratio=0.5,  # MSR을 50%만 적용
                num_deformations=6
            )
            image = random.choice(image)  # 변형된 이미지 중 하나 선택
        
        
        image = self.transform(image)  # 설정된 이미지 변환을 적용합니다.# Albumentations 또는 기타 transform 적용

        if self.is_inference:
            return image
        else:
            target = self.targets[index]  # 해당 이미지의 레이블
            return image, target  # 변환된 이미지와 레이블을 튜플 형태로 반환합니다. 

# Transform Class

In [9]:
class TorchvisionTransform:
    def __init__(self, is_train: bool = True):
        # 공통 변환 설정: 이미지 리사이즈, 텐서 변환, 정규화
        common_transforms = [
            transforms.Resize((224, 224)),  # 이미지를 224x224 크기로 리사이즈
            transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
        ]
        
        if is_train:
            # 훈련용 변환: 랜덤 수평 뒤집기, 랜덤 회전, 색상 조정 추가
            self.transform = transforms.Compose(
                [
                    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 이미지를 수평 뒤집기
                    transforms.RandomRotation(15),  # 최대 15도 회전
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 및 대비 조정
                ] + common_transforms
            )
        else:
            # 검증/테스트용 변환: 공통 변환만 적용
            self.transform = transforms.Compose(common_transforms)

    def __call__(self, image: np.ndarray) -> torch.Tensor:
        image = Image.fromarray(image)  # numpy 배열을 PIL 이미지로 변환
        
        transformed = self.transform(image)  # 설정된 변환을 적용
        
        return transformed  # 변환된 이미지 반환

In [10]:
class UnsharpMask(A.ImageOnlyTransform):
    def __init__(self, kernel_size=5, sigma=1.0, amount=1.0, threshold=0, always_apply=False, p=1.0):
        super(UnsharpMask, self).__init__(always_apply, p)
        self.kernel_size = kernel_size
        self.sigma = sigma
        self.amount = amount
        self.threshold = threshold

    def apply(self, image, **params):
        return self.unsharp_mask(image)

    def unsharp_mask(self, image):
        blurred = cv2.GaussianBlur(image, (self.kernel_size, self.kernel_size), self.sigma)
        sharpened = cv2.addWeighted(image, 1.0 + self.amount, blurred, -self.amount, 0)
        if self.threshold > 0:
            low_contrast_mask = np.absolute(image - blurred) < self.threshold
            np.copyto(sharpened, image, where=low_contrast_mask)
        return sharpened


class AlbumentationsTransform:
    def __init__(self, is_train: bool = True):
        # 공통 변환 설정: 이미지 리사이즈, 정규화, 텐서 변환
        common_transforms = [
            A.Resize(224, 224),  # 이미지를 224x224 크기로 리사이즈
            A.ToGray(p=1.0),  # 그레이스케일 변환
            UnsharpMask(kernel_size=7, sigma=1.5, amount=1.5, threshold=0, p=1.0),  # 언샤프 마스크 적용 # strong
            A.Normalize(mean=[0.5], std=[0.5]),  # 그레이스케일 이미지에 맞는 정규화
            # A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
            ToTensorV2()  # albumentations에서 제공하는 PyTorch 텐서 변환
        ]
        
        if is_train:
            # 훈련용 변환: 랜덤 수평 뒤집기, 랜덤 회전, 랜덤 밝기 및 대비 조정 추가
            self.transform = A.Compose(
                [
                    A.HorizontalFlip(p=0.5),  # 수평 뒤집기
                    A.Rotate(limit=10, p=0.5),  # Rotation (회전 범위 제한)
                    A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.3),  # 랜덤 크롭
                    # A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=(0.2, 0.2), p=0.2),  # 밝기 및 대비 조정
                    A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=0, p=0.9), # 90%확률로 20% 어둡게 # 전반적으로 이미지의 밝기와 채도가 높은 편이라서..!!
                    A.OneOf([A.Emboss(p=0.3), A.Sharpen(p=0.3)], p=0.3),  # Emboss & Sharpen
                    A.GaussianBlur(blur_limit=(3, 5), p=0.3), # 약간의 블러 추가
                    A.CoarseDropout(max_holes=6, max_height=8, max_width=8, p=0.4), # 블럭 추가
                    A.ElasticTransform(alpha=0.3, sigma=10, alpha_affine=5, p=0.3),  # Elastic Transform (강도 조정)       
                    # A.GridDistortion(always_apply=False, p=1, num_steps=1, distort_limit=(-0.03, 0.05), interpolation=2, border_mode=0, value=(0, 0, 0), mask_value=None),                    
                    # A.Affine(scale=(0.95, 1.05), shear=(-3, 3), p=0.5),  # Affine (스케일 및 쉬어 변형)                    
                ] + common_transforms
            )
        else:
            # 검증/테스트용 변환: 공통 변환만 적용
            self.transform = A.Compose(common_transforms)

    def __call__(self, image) -> torch.Tensor:
        # 이미지가 NumPy 배열인지 확인
        if not isinstance(image, np.ndarray):
            raise TypeError("Image should be a NumPy array (OpenCV format).")
        
        # 이미지에 변환 적용 및 결과 반환
        transformed = self.transform(image=image)  # 이미지에 설정된 변환을 적용
        
        return transformed['image']  # 변환된 이미지의 텐서를 반환

In [11]:
class TransformSelector:
    """
    이미지 변환 라이브러리를 선택하기 위한 클래스.
    """
    def __init__(self, transform_type: str):

        # 지원하는 변환 라이브러리인지 확인
        if transform_type in ["torchvision", "albumentations"]:
            self.transform_type = transform_type
        
        else:
            raise ValueError("Unknown transformation library specified.")

    def get_transform(self, is_train: bool):
        
        # 선택된 라이브러리에 따라 적절한 변환 객체를 생성
        if self.transform_type == 'torchvision':
            transform = TorchvisionTransform(is_train=is_train)
        
        elif self.transform_type == 'albumentations':
            transform = AlbumentationsTransform(is_train=is_train)
        
        return transform

# Model Class

In [12]:
class SimpleCNN(nn.Module):
    """
    간단한 CNN 아키텍처를 정의하는 클래스.
    """
    def __init__(self, num_classes: int):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # 순전파 함수 정의
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [13]:
class TorchvisionModel(nn.Module):
    """
    Torchvision에서 제공하는 사전 훈련된 모델을 사용하는 클래스.
    """
    def __init__(
        self, 
        model_name: str, 
        num_classes: int, 
        pretrained: bool
    ):
        super(TorchvisionModel, self).__init__()
        self.model = models.__dict__[model_name](pretrained=pretrained)
        
        # 모델의 최종 분류기 부분을 사용자 정의 클래스 수에 맞게 조정
        if 'fc' in dir(self.model):
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Linear(num_ftrs, num_classes)
        
        elif 'classifier' in dir(self.model):
            num_ftrs = self.model.classifier[-1].in_features
            self.model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        return self.model(x)

In [14]:
class TimmModel(nn.Module):
    """
    Timm 라이브러리를 사용하여 다양한 사전 훈련된 모델을 제공하는 클래스.
    """
    def __init__(
        self, 
        model_name: str, 
        num_classes: int, 
        pretrained: bool
    ):
        super(TimmModel, self).__init__()
        self.model = timm.create_model(
            model_name, 
            pretrained=pretrained, 
            num_classes=num_classes
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        return self.model(x)

In [15]:
class ModelSelector:
    """
    사용할 모델 유형을 선택하는 클래스.
    """
    def __init__(
        self, 
        model_type: str, 
        num_classes: int, 
        **kwargs
    ):
        
        # 모델 유형에 따라 적절한 모델 객체를 생성
        if model_type == 'simple':
            self.model = SimpleCNN(num_classes=num_classes)
        
        elif model_type == 'torchvision':
            self.model = TorchvisionModel(num_classes=num_classes, **kwargs)
        
        elif model_type == 'timm':
            self.model = TimmModel(num_classes=num_classes, **kwargs)
        
        else:
            raise ValueError("Unknown model type specified.")

    def get_model(self) -> nn.Module:

        # 생성된 모델 객체 반환
        return self.model

# Loss Class

In [16]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes: int, smoothing: float = 0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes

    def forward(self, x: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        log_probs = F.log_softmax(x, dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * log_probs, dim=-1))

In [17]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Cross Entropy Loss 계산
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        # 예측 확률 계산
        pt = torch.exp(-ce_loss)
        # Focal Loss 계산
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        elif self.reduction == 'sum':
            return torch.sum(focal_loss)
        else:
            return focal_loss

In [18]:
class Loss(nn.Module):
    """
    모델의 손실함수를 계산하는 클래스.
    """
    def __init__(self):
        super(Loss, self).__init__()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(
        self, 
        outputs: torch.Tensor, 
        targets: torch.Tensor
    ) -> torch.Tensor:
    
        return self.loss_fn(outputs, targets)

# Trainer Class

In [19]:
class Trainer:
    def __init__(
        self, 
        model: nn.Module, 
        device: torch.device, 
        train_loader: DataLoader, 
        val_loader: DataLoader, 
        optimizer: optim.Optimizer,
        scheduler: optim.lr_scheduler,
        loss_fn: torch.nn.modules.loss._Loss, 
        epochs: int,
        result_path: str
    ):
        # 클래스 초기화: 모델, 디바이스, 데이터 로더 등 설정
        self.model = model  # 훈련할 모델
        self.device = device  # 연산을 수행할 디바이스 (CPU or GPU)
        self.train_loader = train_loader  # 훈련 데이터 로더
        self.val_loader = val_loader  # 검증 데이터 로더
        self.optimizer = optimizer  # 최적화 알고리즘
        self.scheduler = scheduler  # 학습률 스케줄러
        self.loss_fn = loss_fn  # 손실 함수
        self.epochs = epochs  # 총 훈련 에폭 수
        self.result_path = result_path  # 모델 저장 경로
        self.best_models = []  # 가장 좋은 상위 3개 모델의 정보를 저장할 리스트
        self.lowest_loss = float('inf')  # 가장 낮은 Loss를 저장할 변수

    def save_model(self, epoch, loss):
        # 모델 저장 경로 설정
        os.makedirs(self.result_path, exist_ok=True)

        # 현재 에폭 모델 저장
        current_model_path = os.path.join(self.result_path, f'model_epoch_{epoch}_loss_{loss:.4f}.pt')
        torch.save(self.model.state_dict(), current_model_path)

        # 최상위 3개 모델 관리
        self.best_models.append((loss, epoch, current_model_path))
        self.best_models.sort()
        if len(self.best_models) > 3:
            _, _, path_to_remove = self.best_models.pop(-1)  # 가장 높은 손실 모델 삭제
            if os.path.exists(path_to_remove):
                os.remove(path_to_remove)

        # 가장 낮은 손실의 모델 저장
        if loss < self.lowest_loss:
            self.lowest_loss = loss
            best_model_path = os.path.join(self.result_path, 'best_model.pt')
            torch.save(self.model.state_dict(), best_model_path)
            print(f"Save {epoch}epoch result. Loss = {loss:.4f}")

    def train_epoch(self) -> float:
        # 한 에폭 동안의 훈련을 진행
        self.model.train()

        total_loss = 0.0
        progress_bar = tqdm(self.train_loader, desc="Training", leave=False)

        for images, targets in progress_bar:
            images, targets = images.to(self.device), targets.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.loss_fn(outputs, targets)
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        return total_loss / len(self.train_loader)

    def validate(self) -> float:
        # 모델의 검증을 진행
        self.model.eval()

        total_loss = 0.0
        progress_bar = tqdm(self.val_loader, desc="Validating", leave=False)

        with torch.no_grad():
            for images, targets in progress_bar:
                images, targets = images.to(self.device), targets.to(self.device)
                outputs = self.model(images)
                loss = self.loss_fn(outputs, targets)
                total_loss += loss.item()
                progress_bar.set_postfix(loss=loss.item())

        return total_loss / len(self.val_loader)

    def train(self) -> float:
        # 전체 훈련 과정을 관리
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}/{self.epochs}")

            train_loss = self.train_epoch()
            val_loss = self.validate()

            current_lr = self.optimizer.param_groups[0]['lr']
            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Learning Rate: {current_lr:.6f}\n")

            self.save_model(epoch, val_loss)
            self.scheduler.step()

        # 학습 완료 후 최종 검증 손실 반환
        return self.lowest_loss

# Model Training

In [20]:
# 학습에 사용할 장비를 선택.
# torch라이브러리에서 gpu를 인식할 경우, cuda로 설정.
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정.
traindata_dir = "./data/train"
traindata_info_file = "./data/train.csv"
save_result_path = "./train_result_code8_nf"

In [22]:
# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
train_info = pd.read_csv(traindata_info_file)

# 총 class의 수를 측정.
num_classes = len(train_info['target'].unique())

In [23]:
# KFold 설정
n_splits = 5  # 5-Fold Cross Validation
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [24]:
# 학습에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
train_transform = transform_selector.get_transform(is_train=True)
val_transform = transform_selector.get_transform(is_train=False)

In [25]:
fold_results = []

In [26]:
# 학습에 사용할 Loss를 선언.
# loss_fn = Loss() #cross_entropy_loss
loss_fn = FocalLoss(alpha=1, gamma=2) #focal_loss
# loss_fn = LabelSmoothingLoss(classes=num_classes, smoothing=0.1) #Label_smoothing_loss

In [27]:
# 스케줄러 초기화
# scheduler_step_size = 15  # 매 15step마다 학습률 감소

In [28]:
# KFold 교차 검증 수행
for fold, (train_idx, val_idx) in enumerate(skf.split(train_info, train_info['target'])):
    print(f'Fold {fold + 1}/{n_splits}')

    # train_df와 val_df를 train_idx와 val_idx로 분할
    train_df = train_info.iloc[train_idx]
    val_df = train_info.iloc[val_idx]
    
    # 클러스터 중심 계산 (train_df의 이미지 사용)
    train_images = [cv2.imread(os.path.join(traindata_dir, path), cv2.IMREAD_COLOR) for path in train_df['image_path'].tolist()]
    cluster_centers = compute_cluster_centers(train_images, n_clusters=150, patch_size=31)


    # 학습에 사용할 Model 선언 (매 Fold마다 모델을 초기화)
    model_selector = ModelSelector(
        model_type='timm', 
        num_classes=num_classes,
        model_name='dm_nfnet_f0', 
        pretrained=True
    )
    model = model_selector.get_model().to(device)

    # optimizer 선언
    optimizer = optim.AdamW(model.parameters(), lr=0.001) # lr=0.0001은 100 epoch일때..

    # 학습에 사용할 Dataset 선언
    train_dataset = CustomDataset(
        root_dir=traindata_dir,
        info_df=train_df,
        transform=train_transform,
        cluster_centers=cluster_centers,  # MSR에 사용할 클러스터 중심
        apply_bpd_msr=True  # BPD와 MSR 적용 활성화
    )
    val_dataset = CustomDataset(
        root_dir=traindata_dir,
        info_df=val_df,
        transform=val_transform,
        cluster_centers=None,  # Validation에서는 MSR을 적용하지 않음
        apply_bpd_msr=False  # BPD와 MSR 적용하지 않음
    )

    # 학습에 사용할 DataLoader 선언
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

    # 한 epoch당 step 수 계산
    steps_per_epoch = len(train_loader)
    
    scheduler_gamma = 0.7  # 학습률을 현재의 70%로 감소 # 0.001 -> 0.0007 -> ..
    epochs = 10
    # StepLR
    # 15 epoch마다 학습률을 감소시키는 스케줄러 선언
    # 5 epoch마다 학습률을 감소시키는 스케줄러 선언
    # decay 25로 해서 lr 안 줄어들게 일단 ㄱ
    epochs_per_lr_decay = 25 # 100 epoch에서 decay 25로 설정했었음
    scheduler_step_size = steps_per_epoch * epochs_per_lr_decay
    
    # 스케줄러 선언
    scheduler = optim.lr_scheduler.StepLR(
        optimizer, 
        step_size=scheduler_step_size, 
        gamma=scheduler_gamma
    )
    
    # #CosineAnnealingLR
    # scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=steps_per_epoch * epochs)

    # Trainer 선언
    trainer = Trainer(
        model=model, 
        device=device, 
        train_loader=train_loader,
        val_loader=val_loader, 
        optimizer=optimizer,
        scheduler=scheduler,
        loss_fn=loss_fn, 
        epochs=epochs,  # 각 fold마다 동일한 epoch수로 학습
        result_path=f"{save_result_path}/fold_{fold + 1}"  # 각 fold 결과 저장
    )

    # 모델 학습 및 결과 저장
    fold_result = trainer.train()
    fold_results.append(fold_result)

# 각 Fold의 결과를 기반으로 평균 성능 계산
average_performance = sum(fold_results) / len(fold_results)
print(f'KFold 평균 성능: {average_performance}')

Fold 1/5
Epoch 1/10


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


# Inference

In [25]:
# 폴드 수 및 모델 저장 경로 설정
n_folds = 5
fold_model_paths = [f"./train_result_code8_nf/fold_{fold + 1}/best_model.pt" for fold in range(n_folds)]

In [26]:
print(fold_model_paths)

['./train_result_code8_nf/fold_1/best_model.pt', './train_result_code8_nf/fold_2/best_model.pt', './train_result_code8_nf/fold_3/best_model.pt', './train_result_code8_nf/fold_4/best_model.pt', './train_result_code8_nf/fold_5/best_model.pt']


In [27]:
# 저장된 모델을 불러와서 앙상블을 수행하는 함수
def ensemble_predict_folds(
    fold_model_paths: list, 
    device: torch.device, 
    test_loader: DataLoader
    ):
    models = []
    
    # 각 폴드의 베스트 모델 불러오기
    for fold_path in fold_model_paths:
        # 모델 초기화 및 로드
        model = ModelSelector(
            model_type='timm', 
            num_classes=num_classes,
            model_name='dm_nfnet_f0', 
            pretrained=False
        ).get_model().to(device)
        model.load_state_dict(torch.load(fold_path, map_location=device))
        model.eval()
        models.append(model)
    
    predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            # 이미지를 GPU 또는 CPU로 이동
            images = images.to(device)
            
            # 폴드별 예측 수행
            fold_preds = []
            for model in models:
                logits = model(images)
                logits = F.softmax(logits, dim=1)  # 확률값으로 변환
                fold_preds.append(logits)
            
            # 폴드별 예측 결과 평균
            avg_preds = torch.mean(torch.stack(fold_preds), dim=0)
            final_preds = avg_preds.argmax(dim=1)
            
            # 예측 결과 저장
            predictions.extend(final_preds.cpu().detach().numpy())
    
    return predictions

In [28]:
# 추론 데이터의 경로와 정보를 가진 파일의 경로를 설정.
testdata_dir = "./data/test"
testdata_info_file = "./data/test.csv"
save_result_path = "./train_result_code8_nf"

In [29]:
# 추론 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
test_info = pd.read_csv(testdata_info_file)

# 총 class 수.
num_classes = 500

In [30]:
# 추론에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
test_transform = transform_selector.get_transform(is_train=False)

# 추론에 사용할 Dataset을 선언.
test_dataset = CustomDataset(
    root_dir=testdata_dir,
    info_df=test_info,
    transform=test_transform,
    is_inference=True
)

# 추론에 사용할 DataLoader를 선언.
test_loader = DataLoader(
    test_dataset, 
    batch_size=64, 
    shuffle=False,
    drop_last=False
)

In [31]:
# 폴드별 저장된 모델을 사용한 앙상블 추론 실행
predictions = ensemble_predict_folds(
    fold_model_paths=fold_model_paths, 
    device=device, 
    test_loader=test_loader
)


  0%|                                                                                                              | 0/157 [00:00<?, ?it/s]


  1%|▋                                                                                                     | 1/157 [00:00<01:04,  2.41it/s]


  1%|█▎                                                                                                    | 2/157 [00:00<01:04,  2.41it/s]


  2%|█▉                                                                                                    | 3/157 [00:01<01:04,  2.38it/s]


  3%|██▌                                                                                                   | 4/157 [00:01<01:04,  2.39it/s]

...


 98%|██████████████████████████████████████████████████████████████████████████████████████████████████  | 154/157 [01:04<00:01,  2.39it/s]


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████▋ | 155/157 [01:04<00:00,  2.41it/s]


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████▎| 156/157 [01:05<00:00,  2.42it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [01:05<00:00,  2.83it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [01:05<00:00,  2.40it/s]

In [32]:
# 모든 클래스에 대한 예측 결과를 하나의 문자열로 합침
test_info['target'] = predictions
test_info = test_info.reset_index().rename(columns={"index": "ID"})
test_info

,ID,image_path,target
0,0,0.JPEG,328
1,1,1.JPEG,414
2,2,2.JPEG,493
3,3,3.JPEG,17
4,4,4.JPEG,388
...,...,...,...
10009,10009,10009.JPEG,235
10010,10010,10010.JPEG,191
10011,10011,10011.JPEG,466
10012,10012,10012.JPEG,400


In [33]:
# 예측 결과를 CSV 파일로 저장
test_info.to_csv("output_code8_nf.csv", index=False)
print(f"추론 결과가 output_code8_nf.csv 파일로 저장되었습니다.")

추론 결과가 output_code8_nf.csv 파일로 저장되었습니다.


In [34]:
# def visualize_gradcam(
#         model: torch.nn.Module,
#         device: torch.device,
#         dataloader: DataLoader,
#         target_layer: str,
#         image_index: int
#     ):

#     # Grad-CAM 추출기를 초기화합니다.
#     cam_extractor = GradCAM(model, target_layer)
    
#     model.eval()  # 모델을 평가 모드로 설정합니다.
#     fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # 시각화를 위한 Figure를 생성합니다.
    
#     # 데이터 로더에서 배치를 반복합니다.
#     current_index = 0
#     for inputs in dataloader:
#         inputs = inputs.to(device)  # 입력 이미지를 장치로 이동합니다.
        
#         outputs = model(inputs)  # 모델을 통해 예측을 수행합니다.
#         _, preds = torch.max(outputs, 1)  # 예측된 클래스 인덱스를 가져옵니다.
        
#         # 배치 내의 각 이미지에 대해 처리합니다.
#         for j in range(inputs.size()[0]):
#             if current_index == image_index:
#                 # CAM을 가져옵니다.
#                 cam = cam_extractor(preds[j].item(), outputs[j].unsqueeze(0))[0]
#                 # CAM을 1채널로 변환합니다.
#                 cam = cam.mean(dim=0).cpu().numpy()
                
#                 # CAM을 원본 이미지 크기로 리사이즈합니다.
#                 cam = cv2.resize(cam, (inputs[j].shape[2], inputs[j].shape[1]))
                
#                 # CAM을 정규화합니다.
#                 cam = (cam - cam.min()) / (cam.max() - cam.min())  # 정규화
#                 ㅈ
#                 # CAM을 0-255 범위로 변환합니다.
#                 cam = np.uint8(255 * cam)
#                 # 컬러맵을 적용하여 RGB 이미지로 변환합니다.
#                 cam = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
#                 cam = cv2.cvtColor(cam, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 변환
                
#                 # 입력 이미지가 1채널 또는 3채널인지 확인하고 처리합니다.
#                 input_image = inputs[j].cpu().numpy().transpose((1, 2, 0))
#                 if input_image.shape[2] == 1:  # 1채널 이미지인 경우
#                     input_image = np.squeeze(input_image, axis=2)  # (H, W, 1) -> (H, W)
#                     input_image = np.stack([input_image] * 3, axis=-1)  # (H, W) -> (H, W, 3)로 변환하여 RGB처럼 만듭니다.
#                 else:  # 3채널 이미지인 경우
#                     input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
#                     input_image = (input_image * 255).astype(np.uint8)  # 정규화된 이미지를 8비트 이미지로 변환합니다.
                
#                 # 오리지널 이미지
#                 axes[0].imshow(input_image)
#                 axes[0].set_title("Original Image")
#                 axes[0].axis('off')
                
#                 # Grad-CAM 이미지
#                 axes[1].imshow(cam)
#                 axes[1].set_title("Grad-CAM Image")
#                 axes[1].axis('off')
                
#                 # 오버레이된 이미지 생성
#                 overlay = cv2.addWeighted(input_image, 0.5, cam, 0.5, 0)
#                 axes[2].imshow(overlay)
#                 axes[2].set_title("Overlay Image")
#                 axes[2].axis('off')
                
#                 plt.show()  # 시각화를 표시합니다.
#                 return
#             current_index += 1

In [35]:
# print(model)

In [36]:
# target_layer = 'layer4.1.act2'

# # Grad-CAM 시각화 실행 (예: 인덱스 3의 이미지를 시각화)

# image_index = 3

# visualize_gradcam(model.model, device, test_loader, target_layer=target_layer, image_index=image_index)